In [2]:
import sys
import os                           # for os interface
import csv
import math
import pickle                       # for full-precision data storage
import numpy as np
import pandas as pd
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict  # append list to disc
import timeit

In [3]:
start_time = timeit.default_timer()
data_file = '../input/itcont.txt'
test_data = '../input/test_data.txt'
header_file = '../input/indiv_header_file.csv'
elapsed = timeit.default_timer() - start_time
print (elapsed)

8.407401037402451e-05


In [4]:
start_time = timeit.default_timer()
# read header line
indiv_header=[]
with open(header_file) as f:
    for line in f:
        header_ele=[ele.strip() for ele in line.split(',') ]
        indiv_header.extend(header_ele) # use extend instead of append
elapsed = timeit.default_timer() - start_time
print (elapsed)
dataset_raw = pd.read_csv(data_file,delimiter='|',\
                      names=indiv_header,keep_default_na=False,dtype=str)
elapsed = timeit.default_timer() - start_time
print (elapsed)

0.0011911229958059266
0.017322304003755562


In [5]:
start_time = timeit.default_timer()
# keep columns of interest
dataset_valid=pd.DataFrame()
dataset_valid=dataset_raw.drop(['AMNDT_IND', 'RPT_TP','TRANSACTION_PGI',\
              'IMAGE_NUM','TRANSACTION_TP','ENTITY_TP',\
              'CITY','STATE','EMPLOYER','OCCUPATION','TRAN_ID',
              'FILE_NUM','MEMO_CD','MEMO_TEXT','SUB_ID'], axis=1)

dataset_valid=dataset_valid[dataset_valid['OTHER_ID'].notnull()]
dataset_valid=dataset_valid[dataset_valid.notnull()]
elapsed = timeit.default_timer() - start_time
print (elapsed)

0.008411861999775283


In [6]:
start_time = timeit.default_timer()
# take the first five characters of 'ZIP_CODE'
# dataset_valid['ZIP_CODE']=dataset_valid['ZIP_CODE'].astype(str).str[0:5]
# 'ZIP_CODE' is valid only when the resulting string length is 5
dataset_valid=dataset_valid[dataset_valid['ZIP_CODE'].str.len()>=5]

# take integer in 'TRANSACTION_AMT'
dataset_valid['TRANSACTION_AMT']=dataset_valid['TRANSACTION_AMT'].astype(int)
# 'TRANSACTION_AMT' is valid only when > 0
dataset_valid=dataset_valid[dataset_valid['TRANSACTION_AMT']>5]

# first : Mark duplicates as True except for the first occurrence.
dataset_repeat=dataset_valid[dataset_valid.duplicated\
                             (['NAME','ZIP_CODE'],keep='first')]

elapsed = timeit.default_timer() - start_time
print (elapsed)

0.051533574995119125


In [7]:
dataset_repeat.iloc[0]

CMTE_ID                  C00394973
NAME               CAFARO, CAPRI S
ZIP_CODE                     44425
TRANSACTION_DT            01102017
TRANSACTION_AMT                396
OTHER_ID                 H4OH14094
Name: 109, dtype: object

In [8]:
start_time = timeit.default_timer()
dataset_arrange=pd.DataFrame()
dataset_arrange['Identity'] = dataset_repeat['CMTE_ID'] + dataset_repeat.ZIP_CODE.str[0:5] + dataset_repeat.TRANSACTION_DT.str[-4:]
dataset_arrange['TRANS_AMT'] = dataset_repeat.TRANSACTION_AMT
dataset_arrange['index1'] = dataset_arrange.index
elapsed = timeit.default_timer() - start_time
print (elapsed)

0.009076114001800306


In [9]:
dataset_arrange.iloc[0:5]

,Identity,TRANS_AMT,index1
109,C00394973444252017,396,109
110,C00394973444252017,25,110
124,C00104083170502017,190,124
208,C00610436995072017,500,208
215,C00161414352432017,125,215


In [12]:
start_time = timeit.default_timer()
grouped=dataset_arrange.groupby('Identity')

grouped.groups
#grouped.groups['C00394973444252017']
#pd.rolling_quantile(grouped['TRANS_AMT'], 2, 0.3)
#s=grouped['TRANS_AMT']
#perc=dataset_arrange.groupby('Identity',as_index=True).expanding(min_periods=1,center=False).quantile(quantile=0.3)
#perc=grouped['TRANS_AMT'].expanding(min_periods=1,center=False).quantile(quantile=0.3)
#total=grouped['TRANS_AMT'].expanding(1).sum()
#times=grouped['TRANS_AMT'].expanding(1).count()


#def func_sum(group):
#    return pd.DataFrame({'sum': group.expanding(1).sum(),'times':group.expanding(1).count()} )
#def func_perc(group):
#    return pd.Series(group.expanding(min_periods=1,center=False).quantile(quantile=0.3))
#def func_total(group):
#    return pd.Series(group.expanding(1).sum())
#def func_times(group):
#    return pd.Series(group.expanding(1).count())

#s1=grouped['TRANS_AMT'].transform(func_perc)
#s2=grouped['TRANS_AMT'].transform(func_total)
#s3=grouped['TRANS_AMT'].transform(func_times)

#pd.concat([dataset_arrange.Identity, s1, s2, s3],names=['Identity','perc','total','times'],axis=1)
#df_1 = dataset_arrange.merge(total)
#result = pd.concat([perc, total], axis=1)
#grouped.groups.keys()
#grouped['TRANS_AMT'].apply(lambda x: x.describe())

def f(group):
    return pd.DataFrame({'Identity': group['Identity'],'Original': group['TRANS_AMT'], 'Perc': group['TRANS_AMT'].expanding(min_periods=1, center=False).quantile(quantile=0.3),\
                         'Total': group['TRANS_AMT'].expanding(1).sum(), 'Times': group['TRANS_AMT'].expanding(1).count()})

X=grouped.apply(f)

elapsed = timeit.default_timer() - start_time
print(elapsed)

0.3605819170043105


In [13]:
grouped.groups

{'C00033555317922017': Int64Index([519], dtype='int64'),
 'C00034157024582017': Int64Index([1671], dtype='int64'),
 'C00034157064612017': Int64Index([1683], dtype='int64'),
 'C00034157115702017': Int64Index([1656], dtype='int64'),
 'C00034157303392017': Int64Index([1641], dtype='int64'),
 'C00034157319012017': Int64Index([1658], dtype='int64'),
 'C00034157319042017': Int64Index([1668, 1714], dtype='int64'),
 'C00034157319062017': Int64Index([1661], dtype='int64'),
 'C00034157334272017': Int64Index([1678], dtype='int64'),
 'C00034157503222017': Int64Index([1636, 1722], dtype='int64'),
 'C00034157572012017': Int64Index([1719], dtype='int64'),
 'C00034157786762017': Int64Index([1726], dtype='int64'),
 'C00034157850272017': Int64Index([1725], dtype='int64'),
 'C00035006945832017': Int64Index([615, 618, 620, 622, 624, 626, 628, 630, 632, 634, 636, 640, 641,
             643, 645, 648, 649, 652, 654, 655, 657, 660, 662, 666, 667, 668,
             673, 674, 676, 679, 681, 684, 686, 687, 689,

In [152]:
start_time = timeit.default_timer()

dd = defaultdict(list)
dataset_final = dataset_arrange.to_dict('records', into=dd)

d1 = defaultdict(list)
d2 = defaultdict(list)
perc = 30.
percent = perc/100.



for item in dataset_final:
    identity=item['Identity']
    amount=item['TRANS_AMT']
    index1=item['index1']
    d1[identity].append(amount)
    if len(d1[identity]) ==1:
        perc_of_cont = amount
        total_cont = amount
        time_of_cont =1
        continue
    else:
        ranked = sorted(d1[identity])
        n = int(round(percent * len(d1[identity])))
        perc_of_cont=ranked[n-1]
        total_cont=sum(d1[identity])
        time_of_cont=len(d1[identity])
    

    d2[index1].append(identity)
    d2[index1].append(perc_of_cont)
    d2[index1].append(total_cont)
    d2[index1].append(time_of_cont)    
    
elapsed = timeit.default_timer() - start_time
print (elapsed)

    
#for k, v in d1.items():       
#    print (k,v)

944.451614868005


In [153]:
#%timeit dataset_final.items()

In [154]:
# group by and sum
# https://stackoverflow.com/questions/33823091/python-pandas-counting-the-frequency-of-a-specific-value-in-each-row-of-datafra
# https://stackoverflow.com/questions/42446006/pandas-find-previous-row-of-matching-value
#data_output['TRANS_AMT_PRE'] = \
#data_output.groupby(['CMTE_ID','ZIP_CODE','TRANS_DT'])['TRANS_AMT'].shift()
#data_output['TRANS_AMT'] + \
#data_output.groupby('ZIP_CODE')['TRANS_AMT'].shift()

In [155]:
#grouped=data_output.groupby(['CMTE_ID','ZIP_CODE','TRANS_DT'])

In [156]:
#new=data_output.groupby(['CMTE_ID']).get_group('C00035006')
#grouped.agg({'TRANS_AMT':np.sum,'TRANS_AMT': np.mean})

In [157]:
#for name, group in grouped:
    #print(name)
#    print(group.TRANS_AMT)

In [158]:
#grouped.bfill